In [1]:
import pandas as pd
import sasoptpy as so
import os
import time

In [2]:
t0 = time.time()

In [3]:
df = pd.read_csv('../data/input.csv')
gk_data = df[df['Pos'] == 'G'].copy().reset_index()
gk_data.set_index('index', inplace=True)

In [4]:
def solve_goalkeeper_problem(gk_data, budget):
    model = so.Model(name='gk_model')
    players = gk_data.index.tolist()
    # Variables
    lineup = model.add_variables(players, name='lineup', vartype=so.binary)
    bench = model.add_variables(players, name='bench', vartype=so.binary)
    # Objective
    total_xp = so.expr_sum(lineup[p] * gk_data.loc[p,'27_Pts'] for p in players) + 0.1 * so.expr_sum(bench[p] * gk_data.loc[p, '27_Pts'] for p in players)
    model.set_objective(-total_xp, name='total_xp_obj', sense='N')
    # Constraints
    model.add_constraints((lineup[p] + bench[p] <= 1 for p in players), name='lineup_or_bench')
    model.add_constraint(so.expr_sum(lineup[p] for p in players) == 1, name='single_lineup')
    model.add_constraint(so.expr_sum(bench[p] for p in players) == 1, name='single_bench')
    model.add_constraint(so.expr_sum((lineup[p] + bench[p]) * gk_data.loc[p, 'BV'] for p in players) <= budget, name='budget_con')
    # Solve Step
    model.export_mps(filename='gk.mps')
    command = 'cbc gk.mps solve solu solution.txt'
    # !{command}
    os.system(command)
    with open('solution.txt', 'r') as f:
        for v in model.get_variables():
            v.set_value(0)
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            var = model.get_variable(words[1])
            var.set_value(float(words[2]))
    print("LINEUP")
    for p in players:
        if lineup[p].get_value() > 0.5:
            print(p, gk_data.loc[p])
    print("BENCH")
    for p in players:
        if bench[p].get_value() > 0.5:
            print(p, gk_data.loc[p])

In [5]:
solve_goalkeeper_problem(gk_data, 9.9)

NOTE: Initialized model gk_model.
LINEUP
522 Pos                   G
Name            Forster
BV                    4
SV                    4
Team        Southampton
27_xMins             81
27_Pts            6.424
Name: 522, dtype: object
BENCH
382 Pos              G
Name        Lloris
BV             5.6
SV             5.6
Team         Spurs
27_xMins        89
27_Pts       4.254
Name: 382, dtype: object


In [6]:
t1 = time.time()
print(t1-t0, 'seconds')

0.3951454162597656 seconds
